# Exercise: Cross-Validation with Symmetric Pair-Input Data

This exercise consists of two tasks. The first task is compulsory: you will not get the right to take the exam if you fail the first task. The second task optional: you do not have to complete the second task but a successful completion will give you an extra point in the exam.

In both tasks, use the K-nearest neighbors classifier with K=1 and Euclidean distance for learning and the concordance index for evaluation. You are encouraged to re-use your own code from the previous exercises. Use the data files `pairs.data`, `features.data`, and `labels.data` that are available in Moodle. The descriptions of these files are provided in the exercise overview, which is also available in Moodle.

Follow the general exercise guidelines of the course (listed in Moodle). Particularly,

- Describe and implement your solution directly to this Jupyter notebook file.
- Remember to describe your solution in general and add detailed comments to the critical parts of your code.
- Remember to justify your design choices and discuss your results.
- Your report must be easy to follow and your code must be runnable in Jupyter notebook.

Feel free to use markdown cells and code cells as you see appropriate.

Submit the finished work to Moodle before the **deadline Monday 18th of February 2019 at 23:59**. Late submissions will be ignored.

## Cover page

DIEGO SANZ VILLAFRUELA

518736

## Task 1 (compulsory)

**You must successfully complete this task in order to get the right to take the exam.**

1. Implement the modified leave-one-out cross-validation scheme that is described in the lecture notes.

2. Estimate and report the generalisation performance of the K-nearest neighbor classifier in predicting the functional similarity of proteins. Use both the unmodified and the modified leave-one-out cross-validation.

3. Discuss your results. In particular, answer the following questions:
 - Why do the two cross-validation schemes produce notably different estimates?
 - Which scheme is appropriate for estimating the generalisation performance on which types of pairs (A, B, or C) and why?

In [6]:
import numpy as np
import math
import operator
import matplotlib.pyplot as plt

%matplotlib inline

In [7]:
"""
Gets samples from CSV file.
"""                
def getSamples(filename, delimiter):
    samples = []
    with open(filename) as f:
        linesIter = iter(f.readlines())

        for line in linesIter:
            line = line.rstrip("\n\r")
            sample = []
            for value in line.split(delimiter):
                sample.append(int(value.upper().replace("P",""))) 
            samples.append(sample)
    return np.array(samples)

In [8]:
"""
Performance measure that indicates how well the model captures the relative ordering/ranking
of the data points.
C-index is measured from 0 to 1, with 0.5 meaning the model wasn't able to capture any information
from the data.
"""
def getCIndex(labels, predictions):
    if labels is None or predictions is None:
        raise Exception("Illegal argument exception")
    if len(labels) != len(predictions):
        raise Exception("The number of labels is not the same to the number of predictions")
        
    size = len(labels)
    h_num = 0
    n = 0
    for i in range(size):
        li = labels[i]
        pi = predictions[i]
        for j in range(i+1,size):
            lj = labels[j]
            pj = predictions[j]
            if ( li != lj):
                n += 1
                if (pi < pj and li < lj) or (pi > pj and li > lj):
                    h_num += 1
                elif pi == pj:
                    h_num += 0.5 # -1 0.5 1
    return h_num/n

In [9]:
"""
Calculates the Ecludean distance between 2 samples. 
It is mainly used when data is continuous.
Note:
Discrete data can only take particular values whereas Continuous data are not restricted
to defined separate values, but can occupy any value over a continuous range.
"""
def euclideanDistance(instance1, instance2, origin, end):
    distance = 0
    for propertyIndex in range(origin, end):
        distance += pow((instance1[propertyIndex] - instance2[propertyIndex]), 2)
    return math.sqrt(distance)

"""
Gets the k closest neighbors from the trainingSet. 
"""
def getNeighbors(trainingSet, testInstance, k, dead_radius, origin_property, end_property):
    distances = []
    
    for train_element in trainingSet:
        euclidean_dist = euclideanDistance(testInstance, train_element, origin_property, end_property)
        distances.append((euclidean_dist, train_element))
           
    distances.sort(key=operator.itemgetter(0))
   
    best_neighbors = []
    n = min(k, len(distances))
    for i in range(n):
        best_neighbors.append(distances[i][1])
    
    return best_neighbors

def getResponse(neighbors):
    length = len(neighbors)
    if length == 0:
        raise ValueError("The number of elements is 0")
    
    prediction = 0
    for neighbor in neighbors:
        prediction += neighbor[-1]
    
    return prediction / len(neighbors)
    
    
"""
Calculates the accuracy of a kNearestNeighbors classifier.
"""
def kNearestNeighbors(trainingData, testData, **kwargs):
    k = kwargs.get("k")
    dead_radius = kwargs.get("radius")
    origin_property = kwargs.get("origin_property")
    end_property = kwargs.get("end_property")
    
    if k is None:
        k = 1
    
    if origin_property is None or end_property is None:
        raise ValueError("You didn't specify the input properties")
        
    if k > len(trainingData):
        raise ValueError("Elements are less than k: {}".format(k))
    
    if testData.ndim != 2:
        raise ValueError("TesData must have dimension 2 and not {}".format(testData.ndim))

    if trainingData.ndim != 2:
        raise ValueError("TrainingData must have dimension 2 and not {}".format(trainingData.ndim))

    predictions = []
    
    for testInstance in testData:
    
        neighbors = getNeighbors(trainingData, testInstance, k, dead_radius, origin_property, end_property)
        
        prediction = getResponse(neighbors)
        
        predictions.append(prediction)
        
    return predictions

In [10]:

def get_filtered_pairs(instances, test_instance, end_property):
    result = []
    
    first_test_element = test_instance[end_property]
    second_test_element = test_instance[end_property + 1]
    
    for instance in instances:
        first_train_element = instance[end_property]
        second_train_element = instance[end_property + 1]
        
        if first_train_element != first_test_element and second_train_element != first_test_element and first_train_element != second_test_element and second_train_element != second_test_element :
            result.append(instance)
    
    return result

def leave_one_out_cv(samples, end, filter_pairs_active, predictionMethod, **kwargs):
    n = len(samples[0])
    all_predictions = []
    all_labels = []
    
    for fold in range(0,len(samples)):
        testData =  samples[fold]
        
        # elements outside the dead zone
        if filter_pairs_active:
            trainingData = get_filtered_pairs(samples[:fold], testData, end) \
                        + get_filtered_pairs(samples[fold+1:], testData, end)
        else:
            trainingData = np.concatenate((samples[:fold],samples[fold+1:]), axis=0)
                
        trainingData = np.array(trainingData)
        # testada must have dimension 2 before being pass to knn
        testData = testData.reshape((1,n))
            
        # -----------------------------------------
        # predictions
        predictions = predictionMethod(trainingData, testData, **kwargs)
        labels = [testInstance[-1] for testInstance in testData]
        
        all_predictions.extend(predictions)
        all_labels.extend(labels)
        #print("Fold:{}, k:{} , len(trainingData:{} , dead_radius:{}".format(fold, k, len(trainingData), dead_radius))
        
    return getCIndex(all_labels, all_predictions)

In [19]:
# READING THE DATA
INPUT_DATA = getSamples('features.data', delimiter=',')
PAIRS_DATA = getSamples('pairs.data', delimiter=',') # np.genfromtxt
OUTPUT_DATA = getSamples('labels.data', delimiter=',')
OUTPUT_DATA = OUTPUT_DATA.reshape((len(OUTPUT_DATA), 1))

ALL_DATA = np.concatenate((INPUT_DATA, PAIRS_DATA, OUTPUT_DATA), axis=1)

k = 1
dead_radius = 1

realistic_index = leave_one_out_cv(ALL_DATA, len(INPUT_DATA[0]), True, kNearestNeighbors, k=k, origin_property=0, end_property=len(INPUT_DATA[0]))
optimistic_index = leave_one_out_cv(ALL_DATA, len(INPUT_DATA[0]), False, kNearestNeighbors, k=k, origin_property=0, end_property=len(INPUT_DATA[0]))

print("Modified leave-one-out cross-validation C-index:", realistic_index)
print("Unmodified leave-one-out cross-validation C-index:", optimistic_index)

Modified leave-one-out cross-validation C-index: 0.6313559322033898
Unmodified leave-one-out cross-validation C-index: 0.7617702448210922


## Why do the two cross-validation schemes produce notably different estimates?

The modified leave-one-out cross-validation gets worse results than the Unmodified leave-one-out cross-validation, 0.6313559322033898 vs 0.7617702448210922.

The reason why in the unmodified leave-one-out cross-validation better obtains better results is that there are dependencies between the training data and the test data, getting optimistic results.

On the other hand, the Unmodified leave-one-out cross-validation gets worse results is because the correlated pair of proteins are eliminated from the train data if they are part of the test data. 

## Which scheme is appropriate for estimating the generalisation performance on which types of pairs (A, B, or C) and why?

The appropriate scheme for estimating the generalisation performance is the modified leave-one-out cross-validation.

As I mentioned above, modified leave-one-out cross-validation eliminates the dependencies between the training data and the test data, making the validation of the model more realistic to the nature of the problem, although lower validation performance(c-index) is obtained.

## Task 2 (optional)

**Successfully completing this task will give you an extra point in the exam.**

1. Design a leave-one-out cross-validation scheme that is appropriate for estimating the generalisation performance on the type of pairs for which the two aforementioned schemes are not appropriate.

2. Explain why your cross-validation scheme is appropriate.

3. Implement your cross-validation scheme. Estimate and report the generalisation performance as in the first task.

4. Discuss your results. In particular, compare the results to those you obtained in the first task and give reasons for any similarities or differences you observe.

*[write your answer to task 2 here]*